In [2]:
import pandas as pd
from geopy.distance import geodesic
from datetime import timedelta
import osmnx as ox
import networkx as nx
import pulp
from datetime import datetime, timedelta

In [103]:
# Координаты бригад и предварительная загрузка данных
brigades = {
    "A": (55.8181190490723, 37.5789909362793),
    "B": (55.7531852722168, 37.4102668762207),
    "C": (55.793628692627, 37.7089691162109),
    "D": (55.6388854980469, 37.7610702514648),
    "D_new": (55.8377838134766, 37.6557426452637)
}

In [163]:
date_of_movement = datetime(2023, 11, 1)


In [164]:
def get_brigade_location(brigade_name, incident_date):
    """
    Возвращает координаты бригады в зависимости от даты происшествия.
    """
    if brigade_name == "D" and incident_date >= date_of_movement:
        return brigades["D_new"]  # Новые координаты после перемещения
    else:
        return brigades[brigade_name]  # Исходные координаты

In [3]:
# Загрузите данные инцидентов
incidents = pd.read_excel('2023.xlsx')

In [5]:
# Список фраз, с которых не должны начинаться записи в столбце "Техописание"
exclude_starts = [
    "BatteryOffline",
    "Требуется замена акб",
    "BatteryPowerOFF",
    "Отключены батареи",
    "Подозрение на кражу АКБ",
    "BatteryLoopOffline"
]

# Фильтрация строк, которые начинаются с указанных фраз
mask = ~incidents['Техописание'].str.startswith(tuple(exclude_starts), na=False)
incidents = incidents[mask]

# обработка данных
incidents['Широта'] = pd.to_numeric(incidents['Широта'], errors='coerce')
incidents['Долгота'] = pd.to_numeric(incidents['Долгота'], errors='coerce')
incidents = incidents.dropna(subset=['Широта', 'Долгота'])

# Сохранение очищенных данных
incidents.to_excel('2023new.xlsx', index=False)

In [159]:
print(incidents)

           № МИ           Т события          STATUS MI  PRIORITY  \
0     M00311886 2023-09-02 02:07:00             Закрыт         3   
1     M00311882 2023-09-02 10:59:00             Закрыт         0   
2     M00311798 2023-09-02 13:38:00             Закрыт         3   
3     M00311684 2023-09-02 14:58:00             Закрыт         3   
4     M00311689 2023-09-02 16:31:00             Закрыт         3   
...         ...                 ...                ...       ...   
1009  M00311674 2023-09-02 04:02:00  Активное ожидание         5   
1010  M00312016 2023-09-02 17:55:00  Активное ожидание         3   
1011  M00316452 2023-10-04 11:48:07             Закрыт         3   
1012  M00313415 2023-09-12 14:10:00             Закрыт         3   
1013  M00318852 2023-10-21 15:16:43             Закрыт         3   

                                       Adress  \
0            Москва, Астрадамская улица, д.7А   
1            Москва, Астрадамская улица, д.7А   
2          Москва, улица Винокурова,

In [105]:
# Обработка данных инцидентов
incidents['Широта'] = pd.to_numeric(incidents['Широта'], errors='coerce')
incidents['Долгота'] = pd.to_numeric(incidents['Долгота'], errors='coerce')
incidents = incidents.dropna(subset=['Широта', 'Долгота'])

In [106]:
# Инициализация графа дорог для Москвы
G = ox.graph_from_place('Moscow, Russia', network_type='drive')
G = ox.project_graph(G)
G = nx.convert_node_labels_to_integers(G)

In [107]:
# Функция для расчета кратчайшего пути
def get_shortest_path_length(G, start_point, end_point):
    start_node = ox.distance.nearest_nodes(G, start_point[1], start_point[0])
    end_node = ox.distance.nearest_nodes(G, end_point[1], end_point[0])
    length = nx.shortest_path_length(G, start_node, end_node, weight='length')
    return length / 1000  # Возвращаем длину в километрах

In [108]:
# Функция для расчета времени в пути
def calculate_travel_time(distance, speed=40):
    return distance / speed  # Время в часах

In [109]:
# Инициализация словаря загрузки бригад
brigade_workload = {brigade: 0 for brigade in brigades.keys()}

In [138]:
# Глобальное определение словаря доступности бригад
brigade_availability = {brigade: pd.Timestamp.min for brigade in brigades.keys()}

def update_brigade_availability(brigade, end_time):
    # Предполагается, что end_time - это datetime объект
    brigade_availability[brigade] = end_time

def some_other_function():
    global brigade_availability

In [139]:
# Функция для обновления загрузки бригад
def update_brigade_workload(brigade, additional_workload):
    brigade_workload[brigade] += additional_workload

In [146]:
def assign_incidents_to_brigades(incidents, brigades, G):
    assignments = []
    incidents_sorted = incidents.sort_values(by='Т события')
    
    for _, incident in incidents_sorted.iterrows():
        incident_coord = (incident['Широта'], incident['Долгота'])
        incident_time = incident['Т события']
        best_brigade = None
        min_delay = pd.Timedelta.max
        
        
        for brigade_name, brigade_coord in brigades.items():
            path_length = get_shortest_path_length(G, brigade_coord, incident_coord)
            travel_time = pd.to_timedelta(calculate_travel_time(path_length), unit='h')

            # Рассчитываем время прибытия, исходя из текущей доступности бригады
            arrival_time = max(brigade_availability[brigade_name], incident_time) + travel_time
            work_end_time = arrival_time + pd.to_timedelta(30, unit='minutes')

            delay = arrival_time - incident_time if arrival_time > incident_time else pd.to_timedelta(0)

            if delay < min_delay:
                best_brigade = brigade_name
                min_delay = delay
                best_end_time = work_end_time

        if best_brigade:
            update_brigade_availability(best_brigade, best_end_time)
            assignments.append({
                'incident_id': incident['№ МИ'],
                'brigade': best_brigade,
                'start_time': arrival_time,
                'end_time': best_end_time
            })

    return assignments

In [151]:
def assign_to_nearest_available_brigade(incidents, brigades, G):
    assignments = []
    sorted_incidents = incidents.sort_values(by='PRIORITY', ascending=False)
    for _, incident in sorted_incidents.iterrows():
        incident_coord = (incident['Широта'], incident['Долгота'])
        incident_start = incident['Т события']
        nearest_brigade, min_distance, estimated_end_time = None, float('inf'), None
        for brigade_name, brigade_coord in brigades.items():
            path_length = get_shortest_path_length(G, brigade_coord, incident_coord)
            travel_time = calculate_travel_time(path_length)
            # Расчет времени начала работы бригады с учетом времени в пути
            start_work_time = incident_start + pd.to_timedelta(travel_time, unit='h')
            # Расчет времени окончания работы (время в пути + 30 минут работы)
            end_work_time = start_work_time + pd.to_timedelta(30, unit='m')
            
            if path_length < min_distance:
                nearest_brigade, min_distance = brigade_name, path_length
                estimated_end_time = end_work_time
        if nearest_brigade is not None:
            update_brigade_availability(nearest_brigade, estimated_end_time)
            update_brigade_workload(nearest_brigade, 1)
            assignments.append((incident['№ МИ'], nearest_brigade, incident_start, estimated_end_time))
    return assignments


In [152]:
# Распределение инцидентов по дням с использованием обновленной логики
def distribute_daily_incidents(incidents, brigades, start_date, end_date, G):
    daily_assignments = {}
    current_date = start_date
    while current_date <= end_date:
        daily_incidents = incidents[incidents['Т события'].dt.date == current_date.date()]
        # Используйте здесь обновленную функцию для назначения инцидентов
        daily_assignments[current_date] = assign_incidents_to_brigades(daily_incidents, brigades, G)
        current_date += timedelta(days=1)
    return daily_assignments


In [155]:
# Сохранения данных в файл
def save_assignments(daily_distributions, incidents):
    output = []
    for day, assignments in daily_distributions.items():
        for assignment in assignments:
            incident_id, brigade, start_time, end_time = assignment
            # Проверка наличия записи перед обращением
            if not incidents[incidents['№ МИ'] == incident_id].empty:
                incident_record = incidents[incidents['№ МИ'] == incident_id].iloc[0]
                output.append({
                    'Date': day.strftime('%Y-%m-%d'),
                    'Incident ID': incident_id,
                    'Assigned Brigade': brigade,
                    'Incident Type': incident_record['Техописание'],
                    # 'Time Assigned': start_time.strftime('%Y-%m-%d %H:%M:%S'),
                    'Time Assigned': incident_record['Т события'].strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(incident_record['Т события']) else 'N/A',
                    'Estimated End Time': end_time.strftime('%Y-%m-%d %H:%M:%S'),
                    'Deadline': incident_record['Крайний срок'].strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(incident_record['Крайний срок']) else 'N/A',

                })
            else:
                print(f"Warning: No incident record found for Incident ID {incident_id}. Skipping.")

    output_df = pd.DataFrame(output)
    output_df.to_csv('daily_assignments_with_details.csv', index=False)

In [156]:
# Вызов функций для распределения и сохранения результатов
start_date = incidents['Т события'].min()
end_date = incidents['Т события'].max()
daily_distributions = distribute_daily_incidents(incidents, brigades, start_date, end_date, G)
save_assignments(daily_distributions, incidents)

In [ ]:
# def assign_incidents_to_brigades(incidents, brigades, G):
#     assignments = []
#     # Отсортируем происшествия по времени начала
#     incidents_sorted = incidents.sort_values(by='Т события')
# 
#     for _, incident in incidents_sorted.iterrows():
#         incident_coord = (incident['Широта'], incident['Долгота'])
#         incident_time = incident['Т события']
#         best_brigade = None
#         smallest_time_delay = timedelta.max  # Инициализируем максимальным значением
#         best_end_work_time = None
# 
#         for brigade_name, brigade_coord in brigades.items():
#             # Расчет времени в пути до места происшествия
#             path_length = get_shortest_path_length(G, brigade_coord, incident_coord)
#             travel_time_hours = calculate_travel_time(path_length)
#             travel_time = timedelta(hours=travel_time_hours)
# 
#             # Планируемое время начала работы бригады учитывая время в пути
#             start_work_time = max(brigade_availability[brigade_name], incident_time) + travel_time
# 
#             # Время окончания работы, включая 30 минут на устранение инцидента
#             end_work_time = start_work_time + timedelta(minutes=30)
# 
#             # Выбираем бригаду с наименьшим временем задержки начала работы
#             time_delay = start_work_time - incident_time
#             if time_delay < smallest_time_delay:
#                 best_brigade = brigade_name
#                 smallest_time_delay = time_delay
#                 best_end_work_time = end_work_time
# 
#         # Обновляем время доступности и нагрузку для выбранной бригады
#         if best_brigade:
#             update_brigade_availability(best_brigade, best_end_work_time)
#             update_brigade_workload(best_brigade, 1)  # Предполагается, что функция update_brigade_workload уже определена
#             assignments.append((incident['№ МИ'], best_brigade, start_work_time, best_end_work_time))
# 
#     return assignments
